<a href="https://colab.research.google.com/github/rickygrosvenor-pramanick/finetuning/blob/main/complete-finetuning/supervised_finetuning_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [14]:
!curl -L -o conll2003.zip https://data.deepai.org/conll2003.zip
!unzip -q conll2003.zip -d conll2003
!ls conll2003

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  959k  100  959k    0     0   581k      0  0:00:01  0:00:01 --:--:--  581k
metadata  test.txt  train.txt  valid.txt


In [10]:
import numpy as np
from datasets import load_dataset
from transformers import (
    BertTokenizerFast,
    BertForTokenClassification,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments
)
from seqeval.metrics import classification_report

MODEL_NAME = "bert-base-uncased"
OUTPUT_DIR = "./bert-ner"

In [19]:
from datasets import Dataset, DatasetDict

def read_conll(path):
    tokens_list, tags_list = [], []
    tokens, tags = [], []
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    tokens_list.append(tokens)
                    tags_list.append(tags)
                    tokens, tags = [], []
            else:
                # format: WORD POS CHUNK NER
                parts = line.split()
                tokens.append(parts[0])
                tags.append(parts[-1])
        # catch last sentence (if no trailing newline)
        if tokens:
            tokens_list.append(tokens)
            tags_list.append(tags)
    return tokens_list, tags_list

# point these at your actual Colab paths
train_tokens, train_tags = read_conll("conll2003/train.txt")
val_tokens,   val_tags   = read_conll("conll2003/valid.txt")
test_tokens,  test_tags  = read_conll("conll2003/test.txt")

train_ds = Dataset.from_dict({"tokens": train_tokens, "ner_tags": train_tags})
val_ds   = Dataset.from_dict({"tokens": val_tokens,   "ner_tags": val_tags})
test_ds  = Dataset.from_dict({"tokens": test_tokens,  "ner_tags": test_tags})

raw_datasets = DatasetDict({
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds
})


In [24]:
train_ds[1]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'ner_tags': ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']}